In [0]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("Test").getOrCreate()

In [0]:
empdata = [(101,"John",30000),(102,"Mary",40000),(103,"Rahul",50000),(104,"Rakesh",60000),(105,"Keshav",70000)]
empdf = spark.createDataFrame(empdata,["id","name","salary"])


In [0]:
deptdf1 = [(101,"IT"),(750,"HR"),(103,"Sales"),(104,"Finance"),(555,"Marketing")]
deptdf = spark.createDataFrame(deptdf1,["id","dept"])

In [0]:
empdf.show()
deptdf.show()

+---+------+------+
| id|  name|salary|
+---+------+------+
|101|  John| 30000|
|102|  Mary| 40000|
|103| Rahul| 50000|
|104|Rakesh| 60000|
|105|Keshav| 70000|
+---+------+------+

+---+---------+
| id|     dept|
+---+---------+
|101|       IT|
|750|       HR|
|103|    Sales|
|104|  Finance|
|555|Marketing|
+---+---------+



In [0]:
from pyspark.sql.functions import broadcast
empdf.join(broadcast(deptdf),how='inner',on='id').show()

+---+------+------+-------+
| id|  name|salary|   dept|
+---+------+------+-------+
|101|  John| 30000|     IT|
|103| Rahul| 50000|  Sales|
|104|Rakesh| 60000|Finance|
+---+------+------+-------+



In [0]:
empdf.join(deptdf.hint("merge"),how='inner',on='id').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   Project [id#13413L, name#13414, salary#13415L, dept#13417]
   +- SortMergeJoin [id#13413L], [id#13416L], Inner
      :- ColumnarToRow
      :  +- PhotonResultStage
      :     +- PhotonSort [id#13413L ASC NULLS FIRST]
      :        +- PhotonShuffleExchangeSource
      :           +- PhotonShuffleMapStage ENSURE_REQUIREMENTS, [id=#9425]
      :              +- PhotonShuffleExchangeSink hashpartitioning(id#13413L, 1024)
      :                 +- PhotonRowToColumnar
      :                    +- LocalTableScan [id#13413L, name#13414, salary#13415L]
      +- ColumnarToRow
         +- PhotonResultStage
            +- PhotonSort [id#13416L ASC NULLS FIRST]
               +- PhotonShuffleExchangeSource
                  +- PhotonShuffleMapStage ENSURE_REQUIREMENTS, [id=#9432]
                     +- PhotonShuffleExchangeSink hashpartitioning(id#13416L, 1024)
                        +- PhotonRowToColumnar
      

In [0]:
empdf.join(deptdf.hint("suffle_hash"),how='inner',on='id').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   ColumnarToRow
   +- PhotonResultStage
      +- PhotonProject [id#13419L, name#13420, salary#13421L, dept#13423]
         +- PhotonBroadcastHashJoin [id#13419L], [id#13422L], Inner, BuildRight, false, true
            :- PhotonRowToColumnar
            :  +- LocalTableScan [id#13419L, name#13420, salary#13421L]
            +- PhotonShuffleExchangeSource
               +- PhotonShuffleMapStage EXECUTOR_BROADCAST, [id=#9501]
                  +- PhotonShuffleExchangeSink SinglePartition
                     +- PhotonRowToColumnar
                        +- LocalTableScan [id#13422L, dept#13423]


== Photon Explanation ==
The query is fully supported by Photon.


In [0]:
empdf.join(deptdf.hint("cartesion")).show()

+---+------+------+---+---------+
| id|  name|salary| id|     dept|
+---+------+------+---+---------+
|101|  John| 30000|101|       IT|
|101|  John| 30000|750|       HR|
|101|  John| 30000|103|    Sales|
|101|  John| 30000|104|  Finance|
|101|  John| 30000|555|Marketing|
|102|  Mary| 40000|101|       IT|
|102|  Mary| 40000|750|       HR|
|102|  Mary| 40000|103|    Sales|
|102|  Mary| 40000|104|  Finance|
|102|  Mary| 40000|555|Marketing|
|103| Rahul| 50000|101|       IT|
|103| Rahul| 50000|750|       HR|
|103| Rahul| 50000|103|    Sales|
|103| Rahul| 50000|104|  Finance|
|103| Rahul| 50000|555|Marketing|
|104|Rakesh| 60000|101|       IT|
|104|Rakesh| 60000|750|       HR|
|104|Rakesh| 60000|103|    Sales|
|104|Rakesh| 60000|104|  Finance|
|104|Rakesh| 60000|555|Marketing|
+---+------+------+---+---------+
only showing top 20 rows


In [0]:
empdf.join(deptdf,empdf.id==deptdf.id,how='inner').show()

+---+------+------+---+-------+
| id|  name|salary| id|   dept|
+---+------+------+---+-------+
|101|  John| 30000|101|     IT|
|103| Rahul| 50000|103|  Sales|
|104|Rakesh| 60000|104|Finance|
+---+------+------+---+-------+

